<a href="https://colab.research.google.com/github/NicolasNevaL/Phyton_Data_Analysis/blob/main/Lenguage_Text_Preprocess_Classification_Booking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre Procesamiento

In [63]:
import re ## Exprexiones regulares
import nltk ## Procesamiento de lenguaje natural
from nltk.corpus import stopwords ## Palabras vacias
from nltk.stem import SnowballStemmer ## Stemming
from nltk.tokenize import word_tokenize ## Tokenizacion
from nltk.tokenize import RegexpTokenizer ## Tokenizacion
from sklearn.feature_extraction.text import CountVectorizer ## Vectorizador
import spacy ## Procesamiento de lenguaje natural

############## Descarga de recursos de nltk ################
nltk.download('punkt') ## Tokenizador
nltk.download('stopwords') ## Palabras vacias
nltk.download('snowball_data') ## Stemming
nltk.download('wordnet') ## Lematizacion

############## Es necesario descargar estos recursos para poder ejecutar el script ################


from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package snowball_data to /root/nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [64]:
import pandas as pd

In [65]:
## Read csv file
df = pd.read_csv('https://raw.githubusercontent.com/Izainea/nlp_ean/refs/heads/main/Datos/Datos%20Crudos/reviews_booking.csv')

<ipython-input-65-948ce0b65ce4>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('https://raw.githubusercontent.com/Izainea/nlp_ean/refs/heads/main/Datos/Datos%20Crudos/reviews_booking.csv')


## Exploratory Analysis

In [66]:
df.head()

,Unnamed: 0,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
0,0,Venezuela,NaN,2 noches,julio de 2023,En pareja,31 de julio de 2023,Excepcional,10,Fue la Mejor Opción que pudimos haber tomado e...,NaN,Sí
1,1,Colombia,NaN,4 noches,octubre de 2024,Persona que viaja sola,23 de octubre de 2024,Fantástica,10,"Ubicación, comodidades y limpieza",NaN,Sí
2,2,Colombia,NaN,1 noche,octubre de 2024,Persona que viaja sola,15 de octubre de 2024,Excepcional,10,"El lugar esta muy bien ubicado, el apartamento...",NaN,Sí
3,3,Colombia,NaN,1 noche,agosto de 2024,En pareja,20 de agosto de 2024,Gran apartamento en una gran ubicación.,10,Todo estuvo perfecto. La ubicación y la relaci...,NaN,Sí
4,4,Colombia,NaN,3 noches,marzo de 2024,En pareja,27 de marzo de 2024,Excepcional,10,"Es muy cómodo y acogedor, la atención es excel...",NaN,Sí


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438213 entries, 0 to 438212
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Unnamed: 0       438213 non-null  int64 
 1   País             436241 non-null  object
 2   Acomodación      19903 non-null   object
 3   Noches           438213 non-null  object
 4   Fecha hospedaje  438213 non-null  object
 5   Grupo viaje      438211 non-null  object
 6   Fecha reseña     438213 non-null  object
 7   Titulo           437552 non-null  object
 8   Calificación     438213 non-null  object
 9   Cosas Positivas  236856 non-null  object
 10  Cosas Negativas  180013 non-null  object
 11  reseña           438213 non-null  object
dtypes: int64(1), object(11)
memory usage: 40.1+ MB


In [68]:
## filter df by 'reseña'
print(df[['reseña']])

       reseña
0          Sí
1          Sí
2          Sí
3          Sí
4          Sí
...       ...
438208     No
438209     No
438210     Sí
438211     No
438212     Sí

[438213 rows x 1 columns]


In [69]:
## group by df[['reseña']]
df.groupby('reseña').count()

,Unnamed: 0,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas
reseña,,,,,,,,,,,
No,189384,188369,3154,189384,189384,189384,189384,188766,189384,0,0
Sí,248829,247872,16749,248829,248829,248827,248829,248786,248829,236856,180013


In [70]:
## if  df['reseña'] == 'Sí' save it in df
df = df[df['reseña'] == 'Sí']
df

,Unnamed: 0,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
0,0,Venezuela,NaN,2 noches,julio de 2023,En pareja,31 de julio de 2023,Excepcional,10,Fue la Mejor Opción que pudimos haber tomado e...,NaN,Sí
1,1,Colombia,NaN,4 noches,octubre de 2024,Persona que viaja sola,23 de octubre de 2024,Fantástica,10,"Ubicación, comodidades y limpieza",NaN,Sí
2,2,Colombia,NaN,1 noche,octubre de 2024,Persona que viaja sola,15 de octubre de 2024,Excepcional,10,"El lugar esta muy bien ubicado, el apartamento...",NaN,Sí
3,3,Colombia,NaN,1 noche,agosto de 2024,En pareja,20 de agosto de 2024,Gran apartamento en una gran ubicación.,10,Todo estuvo perfecto. La ubicación y la relaci...,NaN,Sí
4,4,Colombia,NaN,3 noches,marzo de 2024,En pareja,27 de marzo de 2024,Excepcional,10,"Es muy cómodo y acogedor, la atención es excel...",NaN,Sí
...,...,...,...,...,...,...,...,...,...,...,...,...
438205,438205,Estados Unidos,Apartamento Suite,4 noches,septiembre de 2024,En familia,10 de septiembre de 2024,Como en casa,10,La amabilidad,Todo me gustó,Sí
438206,438206,Colombia,Estudio,1 noche,octubre de 2024,Persona que viaja sola,7 de octubre de 2024,Mal,"3,0",NaN,me cambiaron de hotel y la habitacion que soli...,Sí
438207,438207,Ecuador,Estudio,2 noches,noviembre de 2024,En familia,6 de noviembre de 2024,Pésimo,"1,0",NaN,No me atendieron,Sí
438210,438210,México,Apartamento de 1 dormitorio,4 noches,agosto de 2024,Persona que viaja sola,6 de septiembre de 2024,Pésimo,"1,0",No me pareció nada,No obstante que dos días antes de mi llegada a...,Sí


In [71]:
## convert "," TO "." in df['Calificación']
df['Calificación'] = df['Calificación'].str.replace(',', '.')
df
## convert df['Calificación'] to float  dtype
df['Calificación'] = df['Calificación'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 248829 entries, 0 to 438212
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       248829 non-null  int64  
 1   País             247872 non-null  object 
 2   Acomodación      16749 non-null   object 
 3   Noches           248829 non-null  object 
 4   Fecha hospedaje  248829 non-null  object 
 5   Grupo viaje      248827 non-null  object 
 6   Fecha reseña     248829 non-null  object 
 7   Titulo           248786 non-null  object 
 8   Calificación     248829 non-null  float64
 9   Cosas Positivas  236856 non-null  object 
 10  Cosas Negativas  180013 non-null  object 
 11  reseña           248829 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 24.7+ MB


<ipython-input-71-900ecc780999>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Calificación'] = df['Calificación'].str.replace(',', '.')
<ipython-input-71-900ecc780999>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Calificación'] = df['Calificación'].astype(float)


In [72]:
df.describe()

,Unnamed: 0,Calificación
count,248829.000000,248829.000000
mean,218245.943564,8.203051
std,126795.205827,2.172600
min,0.000000,1.000000
25%,108161.000000,7.000000
50%,218000.000000,9.000000
75%,328428.000000,10.000000
max,438212.000000,10.000000


In [73]:
## print df[Calificación] <= 7 calificación negativa
df[df['Calificación'] <= 7]

,Unnamed: 0,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
12,12,Colombia,NaN,1 noche,septiembre de 2023,Persona que viaja sola,19 de septiembre de 2023,Aceptable,5.0,La atención,Ventilador,Sí
19,19,Colombia,NaN,1 noche,octubre de 2024,En pareja,17 de octubre de 2024,Excelente ubicación,7.0,NaN,Realicé una reserva por bookin y me cobraron v...,Sí
23,23,Colombia,NaN,1 noche,agosto de 2023,Persona que viaja sola,22 de agosto de 2023,Bien,7.0,NaN,no pusierton toallas y no tienen agua caliente,Sí
25,25,Colombia,NaN,1 noche,mayo de 2023,Persona que viaja sola,17 de mayo de 2023,Bien,7.0,"Ubicación, perfecta si vas a hacer alguna dili...","Me cobraron un valor mayor, al que figuraba en...",Sí
32,32,Colombia,NaN,1 noche,septiembre de 2022,Persona que viaja sola,25 de septiembre de 2022,Bien,7.0,Buen desayuno,.,Sí
...,...,...,...,...,...,...,...,...,...,...,...,...
438199,438199,Colombia,Apartamento de 1 dormitorio,8 noches,diciembre de 2023,Persona que viaja sola,26 de diciembre de 2023,Agradable,6.0,"Creo que el ambiente es bastante agradable,aun...",El.problema que tuve con un Duvet blanco que s...,Sí
438202,438202,Colombia,Apartamento de 1 dormitorio,1 noche,septiembre de 2024,Persona que viaja sola,12 de noviembre de 2024,Mal,3.0,NaN,"cucarachas en el apartamento, la anfitriona se...",Sí
438206,438206,Colombia,Estudio,1 noche,octubre de 2024,Persona que viaja sola,7 de octubre de 2024,Mal,3.0,NaN,me cambiaron de hotel y la habitacion que soli...,Sí
438207,438207,Ecuador,Estudio,2 noches,noviembre de 2024,En familia,6 de noviembre de 2024,Pésimo,1.0,NaN,No me atendieron,Sí


In [74]:
## filter df['Cosas Negativas'] by quantity of word
df['Cosas Negativas'].str.split().str.len().describe()

,Cosas Negativas
count,180013.000000
mean,19.448556
std,27.939103
min,1.000000
25%,4.000000
50%,10.000000
75%,24.000000
max,403.000000


In [75]:
## filter df['Cosas Positivas'] by quantity of word
df['Cosas Positivas'].str.split().str.len().describe()

,Cosas Positivas
count,236856.000000
mean,13.006413
std,17.248738
min,1.000000
25%,3.000000
50%,7.000000
75%,16.000000
max,375.000000


In [76]:
df['Cosas Negativas'].str.split().str.len()>=18

,Cosas Negativas
0,False
1,False
2,False
3,False
4,False
...,...
438205,False
438206,True
438207,False
438210,True


In [77]:
# Filtrar por longitud de palabras entre 3 y 16
df_filtered1 = df[(df['Cosas Positivas'].fillna('').str.split().str.len() >= 3) & (df['Cosas Positivas'].fillna('').str.split().str.len() <= 16)]

# Tomar una muestra de 15000 comentarios del DataFrame filtrado
df_sampled1 = df_filtered1.sample(15000)
df_sampled1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 403061 to 292240
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       15000 non-null  int64  
 1   País             14941 non-null  object 
 2   Acomodación      1018 non-null   object 
 3   Noches           15000 non-null  object 
 4   Fecha hospedaje  15000 non-null  object 
 5   Grupo viaje      15000 non-null  object 
 6   Fecha reseña     15000 non-null  object 
 7   Titulo           14998 non-null  object 
 8   Calificación     15000 non-null  float64
 9   Cosas Positivas  15000 non-null  object 
 10  Cosas Negativas  10476 non-null  object 
 11  reseña           15000 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 1.5+ MB


In [78]:
# Filtrar por longitud de palabras entre 1 y 10
df_filtered2 = df[(df['Cosas Negativas'].fillna('').str.split().str.len() >= 1) & (df['Cosas Negativas'].fillna('').str.split().str.len() <= 10)]

# Tomar una muestra de 15000 comentarios del DataFrame filtrado
df_sampled2 = df_filtered2.sample(15000)
df_sampled2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 195011 to 72083
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       15000 non-null  int64  
 1   País             14942 non-null  object 
 2   Acomodación      1089 non-null   object 
 3   Noches           15000 non-null  object 
 4   Fecha hospedaje  15000 non-null  object 
 5   Grupo viaje      15000 non-null  object 
 6   Fecha reseña     15000 non-null  object 
 7   Titulo           14997 non-null  object 
 8   Calificación     15000 non-null  float64
 9   Cosas Positivas  14290 non-null  object 
 10  Cosas Negativas  15000 non-null  object 
 11  reseña           15000 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 1.5+ MB


## Limpieza de texto
## Limpiar BD de comentorios en inglés


In [79]:
## Install aditional libraries
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 83.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [80]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [81]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 50.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [82]:
from sklearn.base import BaseEstimator, TransformerMixin
import spacy


class CleanText(BaseEstimator, TransformerMixin):

  nlp=spacy.load("es_core_news_sm")

    # Stop words
  # stopwords_esp = stopwords.words('spanish') # Moved this initialization to the __init__ method

  def __init__(self):
    self.stopwords_esp = stopwords.words('spanish') # Initialize stopwords_esp here

  def fit(self, X, y=None):
    return self

  def clean_text(self,text):
    if not isinstance(text, str):
      text = str(text)

    # Normalización
    text = text.lower()
    # regex clean
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenización
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in self.stopwords_esp] # Access using self.

    ## Lemmatize

    doc = self.nlp(' '.join(tokens)) # Access using self.
    tokens = [token.lemma_ for token in doc]

    return ' '.join(tokens)

  def transform(self, X, y=None):
    return X.apply(self.clean_text)

  def fit_transform(self, X, y=None):
    return self.transform(X)

cleaner=CleanText()

In [83]:
## Apply Class cleaner (CleanText)
lcp=cleaner.fit_transform(df_sampled1['Cosas Positivas'])

In [84]:
lcn=cleaner.fit_transform(df_sampled2['Cosas Negativas'])

## Frecuencia del Termino y Frecuencia inversa del texto

In [88]:
##################### TF-IDF #####################
# Crear un objeto de la clase TfidfVectorizer

tfidf = TfidfVectorizer()

# Ajustar el objeto a los datos y transformarlos

tfidf_matrix1 = tfidf.fit_transform(lcp)
tfidf_matrix2 = tfidf.fit_transform(lcn)
# Obtener los nombres de las palabras

words = tfidf.get_feature_names_out()

# Crear un DataFrame con los resultados

# tfidf_df = pd.DataFrame(tfidf_matrix1.toarray(), columns=words)
# tfidf_df

In [89]:
# Using a shared vocabulary to solve the inconsistency shape
# Assuming lcp and lcn are the two text datasets

# Combine the text data
combined_text = pd.concat([lcp, lcn], ignore_index=True)

# Create TfidfVectorizer
tfidf = TfidfVectorizer()

# Fit on the combined data
tfidf.fit(combined_text)

# Transform both datasets
tfidf_matrix1 = tfidf.transform(lcp)
tfidf_matrix2 = tfidf.transform(lcn)

# Now, tfidf_matrix1 and tfidf_matrix2 will have the same shape.

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
count_vectorizer=TfidfVectorizer(ngram_range=(2,2))
vectores=count_vectorizer.fit_transform(lcn)
DF_count=pd.DataFrame(vectores.toarray(),columns=count_vectorizer.get_feature_names_out())
DF_count.sum(axis=0).sort_values(ascending=False).head(15)

,0
variedad desayuno,60.156403
agua caliente,53.755125
everything was,43.087514
aire acondicionado,40.369990
habitacin pequea,38.486673
poder mejorar,36.229390
cama duro,29.110638
bao pequeo,27.739363
habitación pequea,27.318338
poco variedad,26.282008


## Random Forest Classification

In [106]:
## Combine the matrices and create labels:
from scipy.sparse import vstack
import numpy as np

# Combine the matrices vertically
X = vstack([tfidf_matrix1, tfidf_matrix2])

# Create labels (0 for negative, 1 for positive)
y = np.concatenate([np.zeros(tfidf_matrix2.shape[0]), np.ones(tfidf_matrix1.shape[0])])

In [ ]:
print("Shape of tfidf_matrix1:", tfidf_matrix1.shape)
print("Shape of tfidf_matrix2:", tfidf_matrix2.shape)

Shape of tfidf_matrix1: (15000, 11200)
Shape of tfidf_matrix2: (15000, 11200)


In [107]:
## Split data into training and testing sets:

from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train the Random Forest classifier:
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=15)

# Train the classifier
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50, random_state=15)

In [ ]:
## Make predictions and evaluate:
from sklearn.metrics import accuracy_score, classification_report

# Make predictions on the testing set
y_pred = rf_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Accuracy: 0.8993333333333333
              precision    recall  f1-score   support

         0.0       0.91      0.89      0.90      3048
         1.0       0.89      0.91      0.90      2952

    accuracy                           0.90      6000
   macro avg       0.90      0.90      0.90      6000
weighted avg       0.90      0.90      0.90      6000



In [ ]:
# Nueva cadena de texto (comentario)
new_comment= " Personal amable, escelente atención personalizada, excelente ubicación, Excelente atención, buena ubicación"

# Limpia y preprocesa la nueva cadena de texto usando la misma función 'clean_text' que usaste antes
# cleaner=CleanText() # Asumiendo que ya tienes el objeto cleaner definido
new_comment_cleaned = cleaner.clean_text(new_comment)

# Transforma la nueva cadena de texto usando el mismo objeto TfidfVectorizer que usaste antes
# tfidf = TfidfVectorizer() # Asumiendo que ya tienes el objeto tfidf definido
new_comment_tfidf = tfidf.transform([new_comment_cleaned])

# Usa el modelo entrenado para predecir la clase de la nueva cadena de texto
prediction = rf_classifier.predict(new_comment_tfidf)

# Imprime la predicción
if prediction[0] == 0:
    print("El comentario es negativo.")
else:
    print("El comentario es positivo.")

ValueError: X has 7078 features, but RandomForestClassifier is expecting 11200 features as input.

In [ ]:
print(new_comment_tfidf)

  (0, 456)	0.26971293149578507
  (0, 845)	0.4432666104167165
  (0, 1478)	0.22986121472606003
  (0, 3910)	0.4448026606572706
  (0, 7703)	0.2049056946997031
  (0, 7712)	0.5361892157598844
  (0, 10379)	0.38807066739568236


In [ ]:
print(tf)

In [ ]:
## Save the Random Forest Model
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define la ruta donde quieres guardar el archivo en Google Drive
# file_path = '/content/drive/MyDrive/Models Machine Learning/rf_classifier_text.joblib'  # Ajusta la ruta según tu necesidad

## Save the model .pk in Google Drive
import joblib
model_path = '/content/drive/My Drive/Models Machine Learning/rf_classifier_text.joblib'  # Specify the path in your Drive
joblib.dump(rf_classifier, model_path)

['/content/drive/My Drive/Models Machine Learning/rf_classifier_text.joblib']

## Support Vector Machine
SVM son robustos para datos de alta dimensión como los que se encuentran en el procesamiento de texto después de la vectorización TF-IDF. Funcionan bien para problemas de clasificación binaria (positivo/negativo) y pueden manejar relaciones no lineales en los datos utilizando diferentes kernels.

In [108]:
from sklearn.svm import SVC
svm_classifier = SVC(kernel='linear') # Puedes cambiar el kernel
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)

In [109]:
# Make predictions on the testing set
y_pred = svm_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Accuracy: 0.8933333333333333
              precision    recall  f1-score   support

         0.0       0.92      0.87      0.89      3048
         1.0       0.87      0.92      0.89      2952

    accuracy                           0.89      6000
   macro avg       0.89      0.89      0.89      6000
weighted avg       0.89      0.89      0.89      6000



In [110]:
# Nueva cadena de texto (comentario)
new_comment= " Personal amable, escelente atención personalizada, excelente ubicación, Excelente atención, buena ubicación"

# Limpia y preprocesa la nueva cadena de texto usando la misma función 'clean_text' que usaste antes
# cleaner=CleanText() # Asumiendo que ya tienes el objeto cleaner definido
new_comment_cleaned = cleaner.clean_text(new_comment)

# Transforma la nueva cadena de texto usando el mismo objeto TfidfVectorizer que usaste antes
# tfidf = TfidfVectorizer() # Asumiendo que ya tienes el objeto tfidf definido
new_comment_tfidf = tfidf.transform([new_comment_cleaned])

# Usa el modelo entrenado para predecir la clase de la nueva cadena de texto
prediction = svm_classifier.predict(new_comment_tfidf)

# Imprime la predicción
if prediction[0] == 0:
    print("El comentario es negativo.")
else:
    print("El comentario es positivo.")

El comentario es negativo.
